In [ ]:
/home/sseyedi/Work/vmCheckNeuroner/test1/runcode.txt

/home/sseyedi/Work/vm2CheckNeuroner/TEST/trained_shadows


SEPERATOR = "aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa aa"
SEPERATOR2=$SEPERATOR" "$SEPERATOR" "$SEPERATOR" "$SEPERATOR" "
for f in *.txt;do echo -ne "\n . \n\n"$SEPERATOR2$SEPERATOR2$SEPERATOR2$SEPERATOR2 >> $f; done

egrep -lr "PATIENT" * >> trainlistwithpatient.txt
sed -e s/xml//g -i trainlistwithpatient.txt 
for f in *;grep -q -F ${f%.*} trainlistwithpatient.txt ||  rm "$file" ;done

a fully connected neural network with one hidden layer of size 64
with ReLU (rectifier linear units) activation functions and a SoftMax layer

In [2]:
import numpy as np

In [3]:
C = np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/Mr1mainSalmanCCCtest.npy")
A = np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/Mr1mainSalmanUnary_scorestest.npy")
Tr= np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/Mr1True_Labels.npy")
OutA = np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/OutMr1mainSalmanUnary_scorestest.npy")
OutC = np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/OutMr1mainSalmanCCCtest.npy")
OutTrue = np.load("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/TEST/OutMr1True_Labels.npy")

In [4]:
Tr.shape,A.shape,C.shape,OutTrue.shape,OutA.shape,OutC.shape

((444178,), (49313,), (49313,), (444178,), (49314,), (49314,))

In [6]:
Tr.shape,A[0].shape,C.shape

((444178,), (30, 93), (49313,))

In [7]:
AA = np.zeros(93).reshape((1,93))
AA.shape
for t in A:
    AA = np.vstack((AA,t))

KeyboardInterrupt: 

In [ ]:
AAA = np.delete(AA,0,axis=0)

In [21]:
p=0
for i in OutA:
    p+=i.shape[0]

In [22]:
i.shape

(10, 93)

In [23]:
Tr.shape

(6162,)

In [24]:
p

444178

In [10]:
AA = np.concatenate(A,axis=0)

In [11]:
AA.shape

(444178, 93)

In [12]:
np.concatenate(C,axis=0).shape

(444178,)

In [ ]:
import numpy as np
train15=np.ones((1,94))
for i in (1,2):#,3,4,5,6,7,8,9,10):
    OutTrue = np.load("outsides/OutMr%sTrue_Labels.npy"%i)
    OutA = np.load("outsides/OutMr%smainSalmanUnary_scorestest.npy"%i,allow_pickle=True)
    OutC = np.load("outsides/OutMr%smainSalmanCCCtest.npy"%i,allow_pickle=True)
    Trues = np.load("originals/Mr%sTrue_Labels.npy"%i)
    A = np.load("originals/Mr%smainSalmanUnary_scorestest.npy"%i,allow_pickle=True)
    C = np.load("originals/Mr%smainSalmanCCCtest.npy"%i,allow_pickle=True)
    #
    C=np.concatenate(C,axis=0)
    OutC = np.concatenate(OutC,axis=0)
    A = np.concatenate(A,axis=0)
    OutA = np.concatenate(OutA,axis=0)
    assert((Trues!=OutTrue).any())
    mask15 = Trues==15
    mask38 = Trues==38
    mask61 = Trues==61
    mask85 = Trues==85
    #
    n15A = np.append(A[mask15],OutA[mask15],axis=0)
    n15C = np.append(C[mask15],OutC[mask15],axis=0)
    n15labels = np.append(np.ones(C[mask15].shape[0]),np.zeros(OutC[mask15].shape[0]))
    n15C = n15C.reshape((n15C.shape[0],1))
    train15=np.append(train15,np.hstack((n15A,n15C)),axis=0)
train15 = np.delete(train15,0,axis=0)
#
OutTrue = np.load("outsides/OutMr%sTrue_Labels.npy"%i)
OutA = np.load("outsides/OutMr%smainSalmanUnary_scorestest.npy"%i,allow_pickle=True)
OutC = np.load("outsides/OutMr%smainSalmanCCCtest.npy"%i,allow_pickle=True)
Trues = np.load("originals/Mr%sTrue_Labels.npy"%i)
A = np.load("originals/Mr%smainSalmanUnary_scorestest.npy"%i,allow_pickle=True)
C = np.load("originals/Mr%smainSalmanCCCtest.npy"%i,allow_pickle=True)
#
C=np.concatenate(C,axis=0)
OutC = np.concatenate(OutC,axis=0)
A = np.concatenate(A,axis=0)
OutA = np.concatenate(OutA,axis=0)
assert((Trues!=OutTrue).any())
mask15 = Trues==15
mask38 = Trues==38
mask61 = Trues==61
mask85 = Trues==85
#
n15A = np.append(A[mask15],OutA[mask15],axis=0)
n15C = np.append(C[mask15],OutC[mask15],axis=0)
testn15labels = np.append(np.ones(C[mask15].shape[0]),np.zeros(OutC[mask15].shape[0]))
n15C = n15C.reshape((n15C.shape[0],1))
test15=np.hstack((n15A,n15C))
#
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
#Wprint(tf.__version__)
#
#train_dataset = tf.data.Dataset.from_tensor_slices((train15,n15labels))
train_ds = tf.data.Dataset.from_tensor_slices(
 (train15, n15labels)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test15, testn15labels)).batch(32)
#
#
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = Dense(64, activation='relu')
        self.d2 = Dense(2, activation='softmax')
    def call(self, x):
        x = self.d1(x)
        return self.d2(x)
#
# Create an instance of the model
model = MyModel()
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD()
#
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')
#
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')
#
@tf.function
def train_step(vectors, labels):
    with tf.GradientTape() as tape:
        predictions = model(vectors)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    #
    train_loss(loss)
    train_accuracy(labels, predictions)
#
#
@tf.function
def test_step(vectors, labels):
    predictions = model(vectors)
    t_loss = loss_object(labels, predictions)
    #
    test_loss(t_loss)
    test_accuracy(labels, predictions)
#
#
EPOCHS = 2
#
for epoch in range(EPOCHS):
    for vectors, labels in train_ds:
        train_step(vectors, labels)
    #
    for test_vectors, test_labels in test_ds:
        test_step(test_vectors, test_labels)
    #
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,\
                        train_loss.result(),\
                        train_accuracy.result()*100,\
                        test_loss.result(),\
                        test_accuracy.result()*100))
    #
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
2019-11-06 14:32:05.051805: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: FMA
2019-11-06 14:32:05.379652: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599960000 Hz
2019-11-06 14:32:05.383203: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x65627a0 executing computations on platform Host. Devices:
2019-11-06 14:32:05.383308: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version


In [10]:
np.loadtxt("RE.+")

OSError: RE.+ not found.

In [5]:
ls

DeId/                                     set2/  testing-PHI-Gold-fixed/
neuronerShadow_dataset_preparation.ipynb  set3/  testing-PHI-Gold-fixed.tar.gz
__pycache__/                              set4/  training-PHI-Gold-Set1/
readme.md                                 set5/  training-PHI-Gold-Set1.tar.gz
README.txt                                set6/  training-PHI-Gold-Set2/
set1/                                     set7/  training-PHI-Gold-Set2.tar.gz
set10/                                    set8/  Untitled.ipynb
set11/                                    set9/  xml_to_brat.py
set12/                                    TEST/
